In [37]:
%pip install simpy
%pip install "numpy<2" "fitter==1.7.1"
import simpy
import random
from fitter import Fitter
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
car_interarrival_data = [
    1.35166493e+00, 1.36167980e-01, 1.84202857e-01, 3.17178141e-01, 2.51452263e+00, 2.65475771e-02,
    1.67443339e-01, 2.09940735e+00, 3.41459398e+00, 1.49252804e-01, 8.74826243e-01, 5.09988862e-02,
    3.67731571e-01, 3.77030083e-01, 8.53583257e-02, 5.31342615e+00, 1.91600101e+00, 1.94930075e-01,
    2.04986668e+00, 2.85673099e+00, 9.75932581e-01, 1.94861521e-01, 6.87801682e-01, 6.63802815e-01,
    2.98096432e+00, 5.37112331e-01, 1.88873714e-01, 1.87810825e+00, 5.43372567e-01, 1.59540088e+00,
    1.35160483e+00, 1.26024903e+00, 3.26581711e-01, 1.90230924e+00, 4.93460497e-01, 1.74663821e-01,
    4.53647701e-01, 4.51542578e-04, 1.38575055e-01, 2.49413797e+00, 4.54583938e-01, 1.85416115e-01,
    3.09104530e-01, 1.59440932e-01, 5.30669164e-01, 1.31789452e+00, 1.63107760e+00, 1.95897103e-01,
    5.18152889e-01, 5.80059344e-01, 2.06985808e+00, 1.55236941e-01, 4.52139818e-01, 1.34950675e+00,
    8.34712429e-01, 1.48393461e-01, 7.90668730e-01, 1.09956407e+00, 2.13323567e-01, 9.73275625e-02,
    3.02919707e+00, 9.79011089e-01, 1.78671931e+00, 6.78908197e-01, 8.11495323e-01, 7.32587321e-01,
    2.12304209e+00, 3.62475404e-02, 4.90550291e-01, 3.93394704e-01, 2.25658935e+00, 8.56790200e-01,
    5.10164918e-01, 1.01885052e-01, 1.92661884e+00, 1.25556899e+00, 1.16785463e-01, 4.46713059e-01,
    1.03879301e+00, 6.22589722e-01, 6.70299754e-02, 1.40772612e+00, 1.43272098e+00, 4.77824425e-02,
    8.21307291e-01, 2.55688529e-02, 1.93705664e-01, 3.02083695e-01, 9.35247162e-01, 1.98320867e+00,
    2.01670473e-01, 7.48256391e-02, 2.13309391e+00, 5.43444933e-01, 6.30951003e-02, 1.63619863e-01,
    1.42022203e+00, 5.78601898e-01, 1.70263363e+00, 1.39347881e-02
]

In [39]:
# 최적의 도착 간격 분포 찾기
f = Fitter(car_interarrival_data, distributions=['uniform', 'norm', 'expon', 'gamma'])
f.fit()

best_distribution = f.get_best()
print("가장 적합한 분포:")
print(best_distribution)

가장 적합한 분포:
{'expon': {'loc': 0.000451542578, 'scale': 0.9064284160297801}}


2025-11-10 05:20:56.284 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted uniform distribution with error=11.54894)
2025-11-10 05:20:56.285 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted norm distribution with error=8.428303)
2025-11-10 05:20:56.295 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted expon distribution with error=4.093185)
2025-11-10 05:20:56.331 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted gamma distribution with error=4.699831)


In [40]:
# 시뮬레이션 파라미터 설정
RANDOM_SEED = 7                # 난수 시드
NUM_MACHINES = 2               # 세차 기계의 수
WASHTIME = 1                   # 세차에 걸리는 시간 (단위: 시간)
SIM_TIME = 20                  # 총 시뮬레이션 시간 (단위: 시간)

random.seed(RANDOM_SEED)       # 난수 시드 설정
Q = []                         # 대기열 상태 저장
B = []                         # 자원 사용 상태 저장

In [41]:
# 세차 작업 정의
def wash(env, car):
    yield env.timeout(WASHTIME)  # 세차 시간 대기

# 고객 프로세스 정의
def car(env, name, carwash, num_machines):
    global Q, B # 그래프 그리기 위해서 적어놓은 것
    print(f'{name}이(가) {env.now:.2f}에 세차장에 도착했습니다.')  # 도착 시점 출력
    arrival_time = env.now # 나중에 대기시간 계산할 때 사용

    # 자원 요청
    req = carwash.request()            # request 객체 생성
    yield req  # 자원 요청 완료까지 대기

    # 대기 시간 기록
    Q.append((round(env.now, 2), len(carwash.queue))) # 대기열 길이 기록, 리스트 뒤에 값을 추가하겠다.
    B.append((round(env.now, 2), carwash.count)) # 자원 사용 상태 기록

    wait_time = env.now - arrival_time  # 대기 시간 계산
    if wait_time > 0:
        print(f'{name}이(가) {env.now:.2f}에 대기 후 세차장에 들어갔습니다.')  # 대기 후 입장
    else:
        print(f'{name}이(가) {env.now:.2f}에 바로 세차장에 들어갔습니다.')  # 바로 입장, <0이니까 시간은 0으로 환산됨.

    # 세차 작업 시작
    yield env.process(wash(env,name))                    # 총 시뮬레이션 시간을 sim_time으로 설정하여 실행  # 세차 프로세스 실행

    print(f'{name}이(가) {env.now:.2f}에 세차장을 떠났습니다.')  # 세차 완료 시점 출력

    # 서비스 완료 후 자원 반납
    carwash.release(req)
    # 대기 시간 기록
    Q.append((round(env.now, 2), len(carwash.queue)))
    B.append((round(env.now, 2), carwash.count))

# 세차장 초기화 및 고객 생성 설정
def setup(env, num_machines, scale):
    carwash = simpy.Resource(env, capacity=num_machines) # 세차 기계 자원 생성
    car_count = 1  # 자동차 번호 초기화

    # 초기 자동차 4대 생성
    for _ in range(4):
        env.process(car(env, f'Car {car_count}', carwash, num_machines))
        car_count += 1  # 자동차 번호 증가

    while True:
        # 지수 분포를 사용하여 도착 간격을 생성
        interarrival_time = random.expovariate(1/ scale)  # scale에 따라 평균 간격 조정, 1/scale 앞에 최적의 scale값을 넣어준거임
        yield env.timeout(interarrival_time)      # interarrival_time만큼 대기
        env.process(car(env, f"Customer {car_count}", carwash, num_machines))  # 고객 프로세스 생성
        car_count += 1  # 자동차 번호 증가

In [42]:
# 지수 분포에서의 scale 값 설정
scale = 0.9064284160297801

In [43]:
print('세차장 시작')
env = simpy.Environment()   # SimPy 환경 생성
env.process(setup(env, NUM_MACHINES, scale))     # 고객 생성 프로세스 추가
env.run(until = SIM_TIME)    # 시뮬레이션 실행

세차장 시작
Car 1이(가) 0.00에 세차장에 도착했습니다.
Car 2이(가) 0.00에 세차장에 도착했습니다.
Car 3이(가) 0.00에 세차장에 도착했습니다.
Car 4이(가) 0.00에 세차장에 도착했습니다.
Car 1이(가) 0.00에 바로 세차장에 들어갔습니다.
Car 2이(가) 0.00에 바로 세차장에 들어갔습니다.
Customer 5이(가) 0.35에 세차장에 도착했습니다.
Customer 6이(가) 0.50에 세차장에 도착했습니다.
Car 1이(가) 1.00에 세차장을 떠났습니다.
Car 2이(가) 1.00에 세차장을 떠났습니다.
Car 3이(가) 1.00에 대기 후 세차장에 들어갔습니다.
Car 4이(가) 1.00에 대기 후 세차장에 들어갔습니다.
Customer 7이(가) 1.46에 세차장에 도착했습니다.
Customer 8이(가) 1.53에 세차장에 도착했습니다.
Car 3이(가) 2.00에 세차장을 떠났습니다.
Car 4이(가) 2.00에 세차장을 떠났습니다.
Customer 5이(가) 2.00에 대기 후 세차장에 들어갔습니다.
Customer 6이(가) 2.00에 대기 후 세차장에 들어갔습니다.
Customer 9이(가) 2.22에 세차장에 도착했습니다.
Customer 10이(가) 2.63에 세차장에 도착했습니다.
Customer 11이(가) 2.69에 세차장에 도착했습니다.
Customer 5이(가) 3.00에 세차장을 떠났습니다.
Customer 6이(가) 3.00에 세차장을 떠났습니다.
Customer 7이(가) 3.00에 대기 후 세차장에 들어갔습니다.
Customer 8이(가) 3.00에 대기 후 세차장에 들어갔습니다.
Customer 12이(가) 3.33에 세차장에 도착했습니다.
Customer 13이(가) 3.36에 세차장에 도착했습니다.
Customer 14이(가) 3.88에 세차장에 도착했습니다.
Customer 15이(가) 3.95에 세차장에 도착했습니다.
Customer 7이(가) 4.00에 세차장을 떠났습니다

In [ ]:
# Q(t) 그래프
times, queue_lengths = zip(*Q)
xticks_times = sorted(set(times + (SIM_TIME,)))
plt.figure(figsize=(10, 5))
plt.step(times, queue_lengths, where='post', marker='o', linestyle='-', color='b')
plt.xlabel('Time')
plt.ylabel('Q')
plt.title('Q(t)')
plt.xticks(xticks_times, [f'{t:.2f}' for t in xticks_times])
plt.yticks(range(0, max(queue_lengths) + 2, 1))
plt.ylim(0, max(queue_lengths) + 1)
plt.xlim(0, SIM_TIME)
plt.grid(False)
plt.show()

In [ ]:
# B(t) 그래프
times_B, resource_states = zip(*B)
xticks_times_B = sorted(set(times_B + (SIM_TIME,)))
plt.figure(figsize=(10, 5))
plt.step(times_B, resource_states, where='post', marker='o', linestyle='-', color='r')
plt.xlabel('Time')
plt.ylabel('B')
plt.title('B(t)')
plt.xticks(xticks_times_B, [f'{t:.2f}' for t in xticks_times_B])
plt.yticks(range(0, NUM_MACHINES + 2, 1))
plt.ylim(0, NUM_MACHINES + 1)
plt.xlim(0, SIM_TIME)
plt.grid(False)
plt.show()

In [5]:
# 프로젝트 1 동질성 검사 
import pandas as pd
from scipy.stats import kruskal

# 엑셀 불러오기 (모든 시트 읽기)
xls = pd.ExcelFile('시뮬 데이터.xlsx')

# 전체 데이터 저장용 리스트
all_data = []

# 모든 시트를 순회하면서 1번, 2번 계산대 데이터를 추출
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    
    # 1번 계산대
    df1 = df.iloc[:, 0:7]  # 왼쪽 영역
    df1.columns = ["순번", "대기열도착시간", "서비스시작시간", "서비스종료시간", "대기시간", "서비스시간", "특이사항"]
    df1["계산대"] = "1번"
    
    # 2번 계산대
    df2 = df.iloc[:, 8:15]  # 오른쪽 영역
    df2.columns = ["순번", "대기열도착시간", "서비스시작시간", "서비스종료시간", "대기시간", "서비스시간", "특이사항"]
    df2["계산대"] = "2번"
    
    all_data.append(pd.concat([df1, df2]))

# 전체 합치기
data = pd.concat(all_data, ignore_index=True)

# 시간 형식을 초 단위로 변환 (예: 0:01:15 → 75초)
def time_to_seconds(t):
    if pd.isnull(t):
        return None
    try:
        h, m, s = map(int, str(t).split(':'))
        return h*3600 + m*60 + s
    except:
        return None

data['대기시간_sec'] = data['대기시간'].apply(time_to_seconds)
data['서비스시간_sec'] = data['서비스시간'].apply(time_to_seconds)

# 각 계산대별로 Kruskal–Wallis 검정
groups = [group['대기시간_sec'].dropna().values for name, group in data.groupby('계산대')]
stat, p = kruskal(*groups)

print(f"대기시간 → H = {stat:.3f}, p = {p:.4f}")
if p < 0.05:
    print("→ 계산대별 대기시간 분포가 다릅니다 (동질성 없음)")
else:
    print("→ 계산대별 대기시간 분포가 같습니다 (동질성 있음)")

# 서비스시간도 동일하게 검정
groups = [group['서비스시간_sec'].dropna().values for name, group in data.groupby('계산대')]
stat, p = kruskal(*groups)

print(f"서비스시간 → H = {stat:.3f}, p = {p:.4f}")
if p < 0.05:
    print("→ 계산대별 서비스시간 분포가 다릅니다 (동질성 없음)")
else:
    print("→ 계산대별 서비스시간 분포가 같습니다 (동질성 있음)")


대기시간 → H = 3.349, p = 0.0672
→ 계산대별 대기시간 분포가 같습니다 (동질성 있음)
서비스시간 → H = 2.581, p = 0.1082
→ 계산대별 서비스시간 분포가 같습니다 (동질성 있음)


In [6]:
# 전체 데이터프레임 크기 (행, 열)
print("전체 shape:", data.shape)

# 상위 5개 행 미리보기
print(data.head())

# 요약(열별 데이터 타입 / 결측치 개수 등)
print(data.info())

# 열별 기본 통계(수치형)
print(data.describe())

전체 shape: (620, 10)
    순번   대기열도착시간   서비스시작시간   서비스종료시간      대기시간     서비스시간  특이사항 계산대  대기시간_sec  \
0  순번    대기열도착시간   서비스시작시간   서비스종료시간      대기시간     서비스시간  특이사항  1번       NaN   
1    1  14:32:36  14:33:47  14:35:02  00:01:11  00:01:15   NaN  1번      71.0   
2    2  14:35:11  14:35:42  14:36:55  00:00:31  00:01:13   NaN  1번      31.0   
3    3  14:37:31  14:37:31  14:38:31  00:00:00  00:01:00   NaN  1번       0.0   
4    4  14:37:59  14:38:31  14:39:09  00:00:32  00:00:38   NaN  1번      32.0   

   서비스시간_sec  
0        NaN  
1       75.0  
2       73.0  
3       60.0  
4       38.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   순번         594 non-null    object 
 1   대기열도착시간    589 non-null    object 
 2   서비스시작시간    589 non-null    object 
 3   서비스종료시간    589 non-null    object 
 4   대기시간       587 non-null    object 
 5   서비스시간      587 non-

In [7]:
# xls = pd.ExcelFile(file_path) 로 불러온 후
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    print(f"시트: {sheet}, 행수: {len(df)}, 열수: {df.shape[1]}")

시트: 표2, 행수: 310, 열수: 19


In [12]:
import pandas as pd

df = pd.read_excel("시뮬 데이터.xlsx")

# 실제 컬럼명 확인
print(df.columns.tolist())

['10/06(월) - 1번 계산대', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', '10/06(월) - 2번 계산대', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']


In [9]:
import pandas as pd

df = pd.read_excel("시뮬 데이터.xlsx")

# 실제 한글 열 이름을 그대로 써야 함
counter1_data = df["순번 1"].dropna().tolist()
counter2_data = df["순번 2"].dropna().tolist()

print("counter1_data = [")
for i, val in enumerate(counter1_data):
    end = ",\n" if (i + 1) % 10 == 0 else ", "
    print(f"    {val:.8e}{end}", end="")
print("]\n")

print("counter2_data = [")
for i, val in enumerate(counter2_data):
    end = ",\n" if (i + 1) % 10 == 0 else ", "
    print(f"    {val:.8e}{end}", end="")
print("]")


KeyError: '순번 1'

In [ ]:
counter1_1006_arrival_time = [
    '14:32:36','14:35:11','14:37:31','14:37:59','14:39:48','14:40:53','14:42:27','14:41:51','14:44:12','14:45:34',
    '14:48:07','14:48:09','14:49:57','14:50:35','14:52:46','14:53:51','14:55:55','14:57:19','14:57:52','15:01:40',
    '15:02:29','15:04:13','15:05:14','15:05:36','15:07:44','15:07:52','15:09:13','15:09:46','15:10:35','15:10:52',
    '15:33:44','15:34:53','15:35:13','15:37:02','15:37:04','15:38:21','15:38:46','15:40:23','15:42:02','15:42:50',
    '15:43:35','15:45:23','15:46:41','15:48:04','15:48:42','15:50:54','15:52:05','15:52:57','15:53:19','15:54:29',
    '15:55:13','15:57:01','15:59:14','15:59:17','16:01:24','16:01:48','16:02:03','16:02:52'
]

counter2_1006_arrival_time = [
    '14:32:48','14:33:25','14:34:58','14:37:24','14:38:15','14:40:11','14:42:48','14:44:02','14:44:54','14:45:23',
    '14:46:23','14:46:54','14:48:57','14:49:48','14:49:57','14:50:10','14:50:17','14:52:16','14:55:42','14:57:17',
    '14:59:38','15:00:18','15:03:59','15:04:58','15:06:32','15:07:53','15:08:54','15:10:29','15:11:23','15:11:57',
    '15:12:01','15:12:57','15:13:11','15:17:32','15:19:07','15:20:35','15:21:04','15:21:07','15:24:29','15:24:39',
    '15:26:42','15:27:40','15:29:11','15:29:17','15:29:35','15:30:30','15:32:51','15:33:09','15:34:10','15:35:07',
    '15:35:16','15:36:08','15:36:30','15:38:03','15:39:15','15:41:30','15:42:09','15:42:14','15:42:19','15:45:38',
    '15:48:17','15:48:30','15:49:50','15:50:02','15:52:51'
]

counter1_1007_arrival_time = [
    '14:34:22','14:36:51','14:37:30','14:39:03','14:39:40','14:43:04','14:43:48','14:45:27','14:46:04','14:46:51',
    '14:48:14','14:49:49','14:50:26','14:51:41','14:54:20','14:54:26','14:56:36','14:58:11','14:58:12','14:58:56',
    '14:59:58','15:00:01','15:01:01','15:01:45','15:02:17','15:02:48','15:03:33','15:03:59','15:05:19','15:07:40',
    '15:07:50','15:08:00','15:09:04','15:09:47','15:11:00','15:11:47','15:12:10','15:15:08','15:15:09','15:17:16',
    '15:18:08','15:18:13','15:19:22','15:20:55','15:22:09','15:23:04','15:23:47','15:24:21','15:24:58','15:26:56',
    '15:29:10','15:30:38','15:32:10','15:32:46','15:34:48','15:35:22','15:35:58','15:36:30','15:37:31','15:39:30',
    '15:39:36','15:40:53','15:43:39','15:45:34','15:45:53','15:47:09','15:47:37','15:48:02','15:49:56','15:50:33',
    '15:51:01','15:54:48','15:55:21','15:55:39','15:56:33'
]


counter2_1007_arrival_time = [
    '14:34:15','14:34:33','14:35:08','14:36:43','14:37:10','14:39:47','14:42:20','14:44:04','14:46:17','14:46:35',
    '14:47:56','14:48:54','14:49:56','14:50:17','14:50:31','14:51:38','14:52:56','14:55:23','14:57:54','15:00:22',
    '15:01:36','15:02:53','15:03:44','15:04:58','15:06:23','15:06:55','15:07:15','15:08:24','15:09:12','15:10:02',
    '15:15:51','15:16:12','15:16:14','15:17:55','15:18:28','15:19:30','15:21:27','15:22:10','15:23:07','15:24:31',
    '15:25:59','15:26:28','15:29:10','15:29:37','15:30:18','15:33:04','15:34:07','15:34:34','15:34:56','15:35:58',
    '15:36:22','15:36:37','15:37:32','15:37:38','15:38:52','15:39:17','15:39:33','15:40:18','15:41:39','15:43:19',
    '15:45:46','15:47:08','15:47:29','15:48:11','15:50:06','15:53:05','15:54:04','15:54:21','15:55:24','15:56:54',
    '15:57:23','15:58:08'
]

counter1_1008_arrival_time = [
    '14:28:30','14:30:05','14:31:30','14:32:25','14:33:27','14:33:59','14:35:25','14:36:21','14:37:20','14:38:16',
    '14:39:40','14:40:29','14:42:58','14:43:03','14:44:54','14:45:14','14:45:30','14:46:12','14:47:30','14:48:00',
    '14:48:48','14:51:43','14:52:13','14:53:06','14:53:52','14:55:29','14:56:02','14:55:39','14:57:54','14:59:05',
    '14:59:47','15:00:31','15:01:21','15:02:08','15:02:48','15:08:03','15:11:01','15:12:02','15:12:48','15:14:04',
    '15:14:31','15:16:11','15:16:58','15:20:52','15:21:34','15:21:35','15:22:30','15:24:39','15:25:08','15:26:10',
    '15:26:11','15:27:27','15:29:10','15:29:30','15:29:44','15:29:57','15:31:20','15:32:05','15:34:20','15:36:01',
    '15:36:32','15:44:03','15:47:11','15:47:38','15:48:07','15:48:45','15:50:26','15:53:23','15:53:46','15:55:07',
    '15:56:00','15:56:29','15:58:59','15:59:01'
]

counter2_1008_arrival_time = [
    '14:29:35','14:30:41','14:31:37','14:32:24','14:34:11','14:35:23','14:34:36','14:36:57','14:38:33','14:38:50',
    '14:40:04','14:40:19','14:43:36','14:43:57','14:45:29','14:45:52','14:47:19','14:48:50','14:49:40','14:50:22',
    '14:51:51','14:51:52','14:52:27','14:53:12','14:55:56','14:56:14','14:56:29','14:56:43','14:58:26','14:58:37',
    '14:59:40','15:02:09','15:03:02','15:03:34','15:04:29','15:09:54','15:11:10','15:12:20','15:15:16','15:17:19',
    '15:18:12','15:18:39','15:20:55','15:21:21','15:23:29','15:25:04','15:26:43','15:27:40','15:29:21','15:29:38',
    '15:29:55','15:31:44','15:32:37','15:33:01','15:34:03','15:35:17','15:36:16','15:37:24','15:38:20','15:39:06',
    '15:39:44','15:40:21','15:40:38','15:42:52','15:43:22','15:43:29','15:47:45','15:49:12','15:49:41','15:51:26',
    '15:52:57','15:54:57','15:57:06','15:57:53','15:59:16'
]


counter1_1015_arrival_time = [
    '15:00:08','15:01:36','15:01:38','15:02:51','15:04:43','15:05:52','15:06:13','15:06:14','15:07:08','15:09:19',
    '15:10:36','15:11:20','15:11:50','15:12:11','15:13:11','15:15:08','15:15:20','15:16:02','15:16:50','15:17:59',
    '15:20:13','15:20:20','15:20:55','15:21:03','15:22:45','15:23:55','15:26:11','15:26:27','15:26:43','15:27:04',
    '15:27:43','15:28:15','15:29:57','15:31:31','15:31:32','15:31:35','15:32:02','15:33:05','15:34:09','15:34:14',
    '15:34:46','15:35:24','15:35:37','15:37:09','15:38:28','15:38:31','15:39:05','15:42:20','15:42:42','15:43:51',
    '15:44:32','15:45:09','15:46:00','15:46:02','15:46:35','15:48:43','15:50:02','15:52:34','15:52:41','15:53:02',
    '15:53:47','15:54:13','15:57:40','15:58:30','16:00:20','16:03:40','16:05:21','16:05:57','16:07:10','16:07:47',
    '16:11:37','16:14:24','16:15:13','16:15:58','16:16:30','16:17:28','16:18:43','16:20:28','16:20:56','16:22:03',
    '16:22:14','16:22:27','16:25:20','16:28:32','16:28:58'
]


counter2_1015_arrival_time= [
    '15:00:55','15:02:30','15:03:36','15:04:07','15:04:40','15:05:16','15:11:33','15:11:49','15:13:49','15:15:18',
    '15:16:08','15:17:28','15:18:11','15:18:58','15:19:17','15:20:33','15:20:58','15:21:21','15:23:13','15:24:37',
    '15:25:10','15:27:10','15:27:42','15:28:36','15:30:07','15:31:04','15:31:15','15:31:35','15:32:26','15:33:33',
    '15:33:58','15:34:07','15:34:15','15:35:02','15:36:16','15:37:35','15:38:31','15:39:04','15:39:38','15:40:00',
    '15:41:18','15:43:07','15:44:11','15:45:19','15:45:44','15:46:02','15:46:19','15:48:02','15:49:37','15:50:40',
    '15:52:36','15:53:00','15:55:30','15:57:09','15:58:46','16:00:01','16:02:30','16:02:40','16:05:37','16:07:30',
    '16:07:41','16:10:36','16:14:44','16:14:56','16:15:40','16:16:25','16:17:01','16:18:33','16:19:41','16:21:06',
    '16:21:46','16:23:01','16:24:54','16:25:07','16:28:16','16:28:41'
]




In [ ]:
counter1_1006_interarrival_time = [155.0, 140.0, 28.0, 109.0, 65.0, 94.0, 36.0, 
                                   141.0, 82.0, 153.0, 2.0, 108.0, 38.0, 131.0, 65.0,
                                     124.0, 84.0, 63.0, 228.0, 49.0, 104.0, 61.0, 22.0,
                                       128.0, 8.0, 81.0, 33.0, 49.0, 17.0, 136.0, 69.0,
                                         20.0, 109.0, 2.0, 77.0, 25.0, 97.0, 99.0, 48.0,
                                           45.0, 108.0, 78.0, 83.0, 38.0, 132.0, 71.0, 52.0, 
                                           22.0, 70.0, 44.0, 108.0, 133.0, 3.0, 127.0, 24.0, 15.0, 49.0]
counter2_1006_interarrival_time = [37, 93, 146, 51, 116, 157, 74, 52, 29, 60, 31, 123, 51, 9, 13, 7, 119, 206, 95, 141, 40,
221, 59, 94, 81, 61, 95, 54, 34, 4, 56, 14, 261, 95, 88, 29, 3, 202, 10, 123, 58, 91, 6, 18,
55, 141, 18, 61, 57, 9, 52, 22, 93, 72, 135, 39, 5, 5, 199, 159, 13, 80, 12, 169]
counter1_1007_interarrival_time = [149, 39, 93, 37, 204, 44, 99, 37, 47, 83, 95, 37, 75, 159, 6, 130, 95, 1, 44, 62,
3, 60, 44, 32, 31, 45, 26, 80, 141, 10, 10, 64, 43, 73, 47, 23, 178, 1, 127, 52,
5, 69, 93, 74, 55, 43, 34, 37, 118, 134, 88, 92, 36, 122, 34, 36, 32, 61, 119, 6,
77, 166, 115, 19, 76, 28, 25, 114, 37, 28, 227, 33, 18, 54]

counter2_1007_interarrival_time = [18, 35, 95, 27, 157, 153, 104, 133, 18, 81, 58, 62, 21, 14, 67, 78, 147, 151, 148, 74,
77, 51, 74, 85, 32, 20, 69, 48, 50, 349, 21, 2, 101, 33, 62, 117, 43, 57, 84,
88, 29, 162, 27, 41, 166, 63, 27, 22, 62, 24, 15, 55, 6, 74, 25, 16, 45, 81, 100,
147, 82, 21, 42, 115, 179, 59, 17, 63, 90, 29, 45]

counter1_1008_interarrival_time = [95, 85, 55, 62, 32, 86, 56, 59, 56, 84, 49, 149, 5, 111, 20, 16, 42, 78, 30, 48,
175, 30, 53, 46, 97, 33, -23, 135, 71, 42, 44, 50, 47, 40, 315, 178, 61, 46, 76,
27, 100, 47, 234, 42, 1, 55, 129, 29, 62, 1, 76, 103, 20, 14, 13, 83, 45, 135,
101, 31, 451, 188, 27, 29, 38, 101, 177, 23, 81, 53, 29, 150, 2]

counter2_1008_interarrival_time = [66, 56, 47, 107, 72, -47, 141, 96, 17, 74, 15, 197, 21, 92, 23, 87, 91, 50, 42, 89,
1, 35, 45, 164, 18, 15, 14, 103, 11, 63, 149, 53, 32, 55, 325, 76, 70, 176, 123,
53, 27, 136, 26, 128, 95, 99, 57, 101, 17, 17, 109, 53, 24, 62, 74, 59, 68, 56,
46, 38, 37, 17, 134, 30, 7, 256, 87, 29, 105, 91, 120, 129, 47, 83]

counter1_1015_interarrival_time = [88, 2, 73, 112, 69, 21, 1, 54, 131, 77, 44, 30, 21, 60, 117, 12, 42, 48, 69, 134,
7, 35, 8, 102, 70, 136, 16, 16, 21, 39, 32, 102, 94, 1, 3, 27, 63, 64, 5, 32,
38, 13, 92, 79, 3, 34, 195, 22, 69, 41, 37, 51, 2, 33, 128, 79, 152, 7, 21,
45, 26, 207, 50, 110, 200, 101, 36, 73, 37, 230, 167, 49, 45, 32, 58, 75, 105,
28, 67, 11, 13, 173, 192, 26]

counter2_1015_interarrival_time = [95, 66, 31, 33, 36, 377, 16, 120, 89, 50, 80, 43, 47, 19, 76, 25, 23, 112, 84, 33,
120, 32, 54, 91, 57, 11, 20, 51, 67, 25, 9, 8, 47, 74, 79, 56, 33, 34, 22, 78,
109, 64, 68, 25, 18, 17, 103, 95, 63, 116, 24, 150, 99, 97, 75, 149, 10, 177, 113,
11, 175, 248, 12, 44, 45, 36, 92, 68, 85, 40, 75, 113, 13, 189, 25]



In [1]:
# 10/06(월) - 1번 계산대 (총 58건)
counter1_1006_arrival_time = [
    '14:32:36','14:35:11','14:37:31','14:37:59','14:39:48','14:40:53','14:42:27','14:41:51','14:44:12','14:45:34',
    '14:48:07','14:48:09','14:49:57','14:50:35','14:52:46','14:53:51','14:55:55','14:57:19','14:57:52','15:01:40',
    '15:02:29','15:04:13','15:05:14','15:05:36','15:07:44','15:07:52','15:09:13','15:09:46','15:10:35','15:10:52',
    '15:33:44','15:34:53','15:35:13','15:37:02','15:37:04','15:38:21','15:38:46','15:40:23','15:42:02','15:42:50',
    '15:43:35','15:45:23','15:46:41','15:48:04','15:48:42','15:50:54','15:52:05','15:52:57','15:53:19','15:54:29',
    '15:55:13','15:57:01','15:59:14','15:59:17','16:01:24','16:01:48','16:02:03','16:02:52'
]


counter1_1006_wait_time = [
    '00:01:11','00:00:31','00:00:00','00:00:32','00:00:00','00:00:11','00:00:00','00:01:21','00:01:01','00:02:17',
    '00:01:39','00:02:26','00:01:26','00:01:33','00:01:04','00:00:56','00:00:00','00:00:00','00:00:28','00:00:00',
    '00:00:00','00:00:11','00:00:09','00:00:53','00:00:00','00:01:00','00:00:31','00:00:26','00:00:24','00:01:12',
    '00:00:00','00:00:00','00:00:48','00:00:56','00:01:59','00:01:30','00:02:02','00:01:18','00:01:03','00:02:05',
    '00:02:28','00:01:55','00:01:30','00:01:31','00:02:34','00:01:19','00:01:28','00:01:44','00:02:40','00:02:57',
    '00:02:51','00:01:40','00:00:02','00:01:51','00:00:40','00:01:37','00:02:33','00:03:27'
]

counter1_1006_service_time = [
    '00:01:15','00:01:13','00:01:00','00:00:38','00:01:16','00:00:42','00:00:45','00:02:01','00:02:38','00:01:55',
    '00:00:49','00:00:48','00:00:45','00:01:42','00:00:57','00:00:41','00:01:13','00:01:01','00:01:32','00:00:34',
    '00:01:55','00:00:59','00:01:06','00:00:50','00:01:08','00:00:52','00:00:28','00:00:47','00:01:05','00:01:17',
    '00:01:05','00:01:08','00:01:57','00:01:05','00:00:48','00:00:57','00:00:53','00:01:24','00:01:50','00:01:08',
    '00:01:15','00:00:53','00:01:24','00:01:41','00:00:57','00:01:20','00:01:08','00:01:18','00:01:27','00:00:38',
    '00:00:37','00:00:35','00:01:52','00:00:56','00:01:21','00:01:11','00:01:43','00:01:19'
]

counter1_1006_service_end_time = [
    '14:35:02','14:36:55','14:38:31','14:39:09','14:41:04','14:41:46','14:43:12','14:45:13','14:47:51','14:49:46',
    '14:50:35','14:51:23','14:52:08','14:53:50','14:54:47','14:55:28','14:57:08','14:58:20','14:59:52','15:02:14',
    '15:04:24','15:05:23','15:06:29','15:07:19','15:08:52','15:09:44','15:10:12','15:10:59','15:12:04','15:13:21',
    '15:34:49','15:36:01','15:37:58','15:39:03','15:39:51','15:40:48','15:41:41','15:43:05','15:44:55','15:46:03',
    '15:47:18','15:48:11','15:49:35','15:51:16','15:52:13','15:53:33','15:54:41','15:55:59','15:57:26','15:58:04',
    '15:58:41','15:59:16','16:01:08','16:02:04','16:03:25','16:04:36','16:06:19','16:07:38'
]

# 10/06(월) - 2번 계산대 (총 65건)
counter2_1006_arrival_time = [
    '14:32:48','14:33:25','14:34:58','14:37:24','14:38:15','14:40:11','14:42:48','14:44:02','14:44:54','14:45:23',
    '14:46:23','14:46:54','14:48:57','14:49:48','14:49:57','14:50:10','14:50:17','14:52:16','14:55:42','14:57:17',
    '14:59:38','15:00:18','15:03:59','15:04:58','15:06:32','15:07:53','15:08:54','15:10:29','15:11:23','15:11:57',
    '15:12:01','15:12:57','15:13:11','15:17:32','15:19:07','15:20:35','15:21:04','15:21:07','15:24:29','15:24:39',
    '15:26:42','15:27:40','15:29:11','15:29:17','15:29:35','15:30:30','15:32:51','15:33:09','15:34:10','15:35:07',
    '15:35:16','15:36:08','15:36:30','15:38:03','15:39:15','15:41:30','15:42:09','15:42:14','15:42:19','15:45:38',
    '15:48:17','15:48:30','15:49:50','15:50:02','15:52:51'
]
counter2_1006_wait_time = [
    '00:00:54','00:00:55','00:01:15','00:00:10','00:00:08','00:00:13','00:00:08','00:00:28','00:00:10','00:00:22',
    '00:01:30','00:01:13','00:00:22','00:00:46','00:00:54','00:01:11','00:01:40','00:00:20','00:00:15','00:00:29',
    '00:00:13','00:00:35','00:00:15','00:00:42','00:00:16','00:00:27','00:00:55','00:00:09','00:00:07','00:00:37',
    '00:01:47','00:02:04','00:02:45','00:00:22','00:00:17','00:00:13','00:00:25','00:01:35','00:00:09','00:00:41',
    '00:00:35','00:00:16','00:00:10','00:00:50','00:01:19','00:01:28','00:00:47','00:01:12','00:00:47','00:00:40',
    '00:01:59','00:01:54','00:02:56','00:02:19','00:01:41','00:00:37','00:02:11','00:02:40','00:03:50','00:01:07',
    '00:00:18','00:00:56','00:02:00','00:03:08','00:01:08'
]

counter2_1006_service_time = [
    '00:00:37','00:00:19','00:00:19','00:00:21','00:00:28','00:00:53','00:00:34','00:00:05','00:00:29','00:01:57',
    '00:00:09','00:01:08','00:00:59','00:00:10','00:00:15','00:00:25','00:00:26','00:00:46','00:00:47','00:00:22',
    '00:00:51','00:00:51','00:01:23','00:00:35','00:01:20','00:00:49','00:00:30','00:00:40','00:00:58','00:00:50',
    '00:01:08','00:00:49','00:00:34','00:00:43','00:01:01','00:00:32','00:01:01','00:00:38','00:00:34','00:01:33',
    '00:00:21','00:01:12','00:00:40','00:00:45','00:00:55','00:01:28','00:00:24','00:00:21','00:00:35','00:01:09',
    '00:00:29','00:00:48','00:00:29','00:00:25','00:00:26','00:02:08','00:00:20','00:01:00','00:00:21','00:00:32',
    '00:00:24','00:02:09','00:01:14','00:00:18','00:00:38'
]

counter2_1006_service_end_time = [
    '14:34:19','14:34:39','14:36:32','14:37:55','14:38:51','14:41:17','14:43:30','14:44:35','14:45:33','14:47:42',
    '14:48:02','14:49:15','14:50:18','14:50:44','14:51:06','14:51:46','14:52:23','14:53:22','14:56:44','14:58:08',
    '15:00:42','15:01:44','15:05:37','15:06:15','15:08:08','15:09:09','15:10:19','15:11:18','15:12:28','15:13:24',
    '15:14:56','15:15:50','15:16:30','15:18:37','15:20:25','15:21:20','15:22:30','15:23:20','15:25:12','15:26:53',
    '15:27:38','15:29:08','15:30:01','15:30:52','15:31:49','15:33:26','15:34:02','15:34:42','15:35:32','15:36:56',
    '15:37:44','15:38:50','15:39:55','15:40:47','15:41:22','15:44:15','15:44:40','15:45:54','15:46:30','15:47:17',
    '15:48:59','15:51:35','15:53:04','15:53:28','15:54:37'
]


# 10/07(화) - 1번 계산대 (총 74건, 38번 제외)

counter1_1007_arrival_time = [
    '14:34:22','14:36:51','14:37:30','14:39:03','14:39:40','14:43:04','14:43:48','14:45:27','14:46:04','14:46:51',
    '14:48:14','14:49:49','14:50:26','14:51:41','14:54:20','14:54:26','14:56:36','14:58:11','14:58:12','14:58:56',
    '14:59:58','15:00:01','15:01:01','15:01:45','15:02:17','15:02:48','15:03:33','15:03:59','15:05:19','15:07:40',
    '15:07:50','15:08:00','15:09:04','15:09:47','15:11:00','15:11:47','15:12:10','15:15:08','15:15:09','15:17:16',
    '15:18:08','15:18:13','15:19:22','15:20:55','15:22:09','15:23:04','15:23:47','15:24:21','15:24:58','15:26:56',
    '15:29:10','15:30:38','15:32:10','15:32:46','15:34:48','15:35:22','15:35:58','15:36:30','15:37:31','15:39:30',
    '15:39:36','15:40:53','15:43:39','15:45:34','15:45:53','15:47:09','15:47:37','15:48:02','15:49:56','15:50:33',
    '15:51:01','15:54:48','15:55:21','15:55:39','15:56:33'
]

counter1_1007_wait_time = [
    '00:01:04','00:00:38','00:00:49','00:00:00','00:00:31','00:00:00','00:00:00','00:00:09','00:00:35','00:01:00',
    '00:01:33','00:01:37','00:01:24','00:01:02','00:01:36','00:02:04','00:00:28','00:00:00','00:00:45','00:00:31',
    '00:00:01','00:00:43','00:00:02','00:00:00','00:00:00','00:00:09','00:00:56','00:01:42','00:01:47','00:00:16',
    '00:01:06','00:02:08','00:02:15','00:02:50','00:02:00','00:01:37','00:01:49',
    '00:00:52','00:00:00','00:00:23','00:00:43','00:00:36','00:00:52','00:00:09','00:00:50','00:01:26','00:01:54',
    '00:01:59','00:00:38','00:00:00','00:00:02','00:00:00','00:00:10','00:00:04','00:01:09','00:00:57','00:00:48',
    '00:01:11','00:02:50','00:03:04','00:02:08','00:01:17','00:00:38','00:00:57','00:00:29','00:01:21','00:03:08',
    '00:02:11','00:02:10','00:02:47','00:00:20','00:01:06','00:01:38','00:01:37'
]

counter1_1007_service_time = [
    '00:02:03','00:00:50','00:00:43','00:01:08','00:01:21','00:00:32','00:01:48','00:01:03','00:01:11','00:01:56',
    '00:01:39','00:00:24','00:00:53','00:03:13','00:00:24','00:00:34','00:01:06','00:00:45','00:00:30','00:00:31',
    '00:00:45','00:00:17','00:00:30','00:00:26','00:00:40','00:01:32','00:01:11','00:01:22','00:00:49','00:00:59',
    '00:01:12','00:01:11','00:01:18','00:00:23','00:00:24','00:00:34','00:01:08',
    '00:01:14','00:01:15','00:00:24','00:01:01','00:01:49','00:00:31','00:00:35','00:01:18','00:01:01','00:00:41',
    '00:00:37','00:00:59','00:01:28','00:00:58','00:00:44','00:01:55','00:01:38','00:00:23','00:00:23','00:01:23',
    '00:03:36','00:00:19','00:00:20','00:01:54','00:01:16','00:00:36','00:00:47','00:01:17','00:02:12','00:00:56',
    '00:00:35','00:01:05','00:01:18','00:00:43','00:00:48','00:00:52','00:01:08'
]

counter1_1007_service_end_time = [
    '14:37:29','14:38:19','14:39:02','14:40:11','14:41:32','14:43:36','14:45:36','14:46:39','14:47:50','14:49:47',
    '14:51:26','14:51:50','14:52:43','14:55:56','14:56:20','14:57:04','14:58:10','14:58:56','14:59:27','14:59:58',
    '15:00:44','15:01:01','15:01:33','15:02:11','15:02:57','15:04:29','15:05:40','15:07:03','15:07:55','15:08:55',
    '15:10:08','15:11:19','15:12:37','15:13:00','15:13:24','15:13:58','15:15:07',
    '15:17:15','15:18:31','15:18:55','15:19:57','15:21:47','15:22:18','15:22:53','15:25:12','15:26:14','15:26:56',
    '15:27:34','15:28:33','15:30:38','15:31:38','15:32:54','15:34:51','15:36:30','15:36:54','15:37:18','15:38:41',
    '15:42:18','15:42:39','15:43:00','15:44:55','15:46:12','15:46:48','15:47:37','15:48:55','15:51:10','15:52:06',
    '15:52:42','15:53:48','15:55:06','15:55:51','15:57:15','15:58:09','15:59:18'
]

# 10/07(화) - 2번 계산대 (총 72건)

counter2_1007_arrival_time = [
    '14:34:15','14:34:33','14:35:08','14:36:43','14:37:10','14:39:47','14:42:20','14:44:04','14:46:17','14:46:35',
    '14:47:56','14:48:54','14:49:56','14:50:17','14:50:31','14:51:38','14:52:56','14:55:23','14:57:54','15:00:22',
    '15:01:36','15:02:53','15:03:44','15:04:58','15:06:23','15:06:55','15:07:15','15:08:24','15:09:12','15:10:02',
    '15:15:51','15:16:12','15:16:14','15:17:55','15:18:28','15:19:30','15:21:27','15:22:10','15:23:07','15:24:31',
    '15:25:59','15:26:28','15:29:10','15:29:37','15:30:18','15:33:04','15:34:07','15:34:34','15:34:56','15:35:58',
    '15:36:22','15:36:37','15:37:32','15:37:38','15:38:52','15:39:17','15:39:33','15:40:18','15:41:39','15:43:19',
    '15:45:46','15:47:08','15:47:29','15:48:11','15:50:06','15:53:05','15:54:04','15:54:21','15:55:24','15:56:54',
    '15:57:23','15:58:08'
]

counter2_1007_wait_time = [
    '00:00:00','00:01:22','00:01:23','00:01:10','00:02:49','00:01:29','00:01:12','00:00:48','00:00:00','00:00:43',
    '00:00:41','00:00:23','00:00:20','00:00:58','00:01:37','00:01:24','00:01:16','00:00:00','00:00:00','00:00:00',
    '00:00:13','00:00:58','00:01:17','00:02:19','00:01:48','00:02:17','00:03:26','00:03:39','00:03:59','00:03:55',
    '00:00:23','00:01:26','00:02:15','00:01:36','00:02:31','00:02:27','00:01:21','00:01:44','00:01:58','00:01:17',
    '00:01:13','00:01:47','00:00:01','00:01:07','00:02:09','00:00:10','00:00:42','00:01:24','00:01:56','00:01:51',
    '00:02:42','00:03:11','00:03:00','00:03:47','00:03:50','00:04:15','00:04:43','00:04:30','00:04:06','00:03:52',
    '00:03:00','00:02:58','00:03:54','00:03:58','00:02:57','00:00:46','00:00:55','00:01:53','00:02:10','00:01:19',
    '00:01:33','00:01:56'
]

counter2_1007_service_time = [
    '00:01:40','00:00:36','00:01:22','00:02:06','00:01:17','00:02:16','00:01:20','00:00:40','00:01:01','00:01:19',
    '00:00:40','00:00:59','00:00:59','00:00:53','00:00:54','00:01:10','00:01:10','00:01:56','00:01:17','00:01:27',
    '00:02:02','00:01:10','00:02:16','00:00:54','00:01:01','00:01:29','00:01:22','00:01:08','00:00:46','00:02:17',
    '00:01:24','00:00:51','00:01:02','00:01:28','00:00:58','00:00:51','00:01:06','00:01:11','00:00:43','00:01:24',
    '00:01:03','00:00:56','00:01:33','00:01:43','00:00:47','00:01:35','00:01:09','00:00:54','00:00:57','00:01:15',
    '00:00:44','00:00:44','00:01:26','00:01:17','00:00:50','00:00:44','00:00:32','00:00:57','00:01:26','00:01:35',
    '00:01:20','00:01:17','00:00:46','00:00:54','00:00:48','00:01:08','00:01:15','00:01:20','00:00:39','00:00:43',
    '00:01:08','00:01:07'
]

counter2_1007_service_end_time = [
    '14:35:55','14:36:31','14:37:53','14:39:59','14:41:16','14:43:32','14:44:52','14:45:32','14:47:18','14:48:37',
    '14:49:17','14:50:16','14:51:15','14:52:08','14:53:02','14:54:12','14:55:22','14:57:19','14:59:11','15:01:49',
    '15:03:51','15:05:01','15:07:17','15:08:11','15:09:12','15:10:41','15:12:03','15:13:11','15:13:57','15:16:14',
    '15:17:38','15:18:29','15:19:31','15:20:59','15:21:57','15:22:48','15:23:54','15:25:05','15:25:48','15:27:12',
    '15:28:15','15:29:11','15:30:44','15:32:27','15:33:14','15:34:49','15:35:58','15:36:52','15:37:49','15:39:04',
    '15:39:48','15:40:32','15:41:25','15:42:42','15:43:32','15:44:16','15:44:48','15:45:45','15:47:11','15:48:46',
    '15:50:06','15:51:23','15:52:09','15:53:03','15:53:51','15:54:59','15:56:14','15:57:34','15:58:13','15:58:56',
    '16:00:04','16:01:11'
]


# 10월 8일 (수) 1번 계산대(74개)

counter1_1008_service_interarrival_time = [
    '14:28:30','14:30:05','14:31:30','14:32:25','14:33:27','14:33:59','14:35:25','14:36:21','14:37:20','14:38:16',
    '14:39:40','14:40:29','14:42:58','14:43:03','14:44:54','14:45:14','14:45:30','14:46:12','14:47:30','14:48:00',
    '14:48:48','14:51:43','14:52:13','14:53:06','14:53:52','14:55:29','14:56:02','14:55:39','14:57:54','14:59:05',
    '14:59:47','15:00:31','15:01:21','15:02:08','15:02:48','15:08:03','15:11:01','15:12:02','15:12:48','15:14:04',
    '15:14:31','15:16:11','15:16:58','15:20:52','15:21:34','15:21:35','15:22:30','15:24:39','15:25:08','15:26:10',
    '15:26:11','15:27:27','15:29:10','15:29:30','15:29:44','15:29:57','15:31:20','15:32:05','15:34:20','15:36:01',
    '15:36:32','15:44:03','15:47:11','15:47:38','15:48:07','15:48:45','15:50:26','15:53:23','15:53:46','15:55:07',
    '15:56:00','15:56:29','15:58:59','15:59:01'
]



counter1_1008_wait_times = [
    '00:00:00','00:00:20','00:00:00','00:00:17','00:00:25','00:01:03','00:00:25','00:00:00','00:00:00','00:00:14',
    '00:00:24','00:00:23','00:00:00','00:00:22','00:00:00','00:00:26','00:00:32','00:00:45','00:00:32','00:02:06',
    '00:02:15','00:00:27','00:00:59','00:00:24','00:00:52','00:00:41','00:01:21','00:03:03','00:01:42','00:01:14',
    '00:01:05','00:01:02','00:01:36','00:02:02','00:02:14','00:00:00','00:00:00','00:00:00','00:00:04','00:00:00',
    '00:00:53','00:00:00','00:00:06','00:00:00','00:00:33','00:01:19','00:01:34','00:00:15','00:00:31','00:00:00',
    '00:01:24','00:00:35','00:00:00','00:00:32','00:00:36','00:01:15','00:01:11','00:02:07','00:01:45','00:07:01',
    '00:07:41','00:01:52','00:00:00','00:00:17','00:00:47','00:01:13','00:00:52','00:00:00','00:00:14','00:00:00',
    '00:00:07','00:00:56','00:00:00','00:00:14'
]


counter1_1008_service_times = [
    '00:01:55','00:00:55','00:01:12','00:01:10','00:01:10','00:00:48','00:00:20','00:00:24','00:01:10','00:01:34',
    '00:00:48','00:01:13','00:00:27','00:00:25','00:00:46','00:00:22','00:00:55','00:01:05','00:02:04','00:00:57',
    '00:01:07','00:01:02','00:00:18','00:01:14','00:01:26','00:01:13','00:01:19','00:00:54','00:00:43','00:00:33',
    '00:00:41','00:01:24','00:01:13','00:00:52','00:01:02','00:02:33','00:00:25','00:00:50','00:01:08','00:01:20',
    '00:00:37','00:00:53','00:02:32','00:01:15','00:00:47','00:01:10','00:00:50','00:00:45','00:00:23','00:01:25',
    '00:00:27','00:00:31','00:00:51','00:00:18','00:00:52','00:01:19','00:01:41','00:01:53','00:06:57','00:01:12',
    '00:01:42','00:00:44','00:00:44','00:00:59','00:01:04','00:01:20','00:00:30','00:00:37','00:00:46','00:01:00',
    '00:01:18','00:00:58','00:00:16','00:01:43'
]


counter1_1008_service_end_times = [
    '14:30:25','14:31:20','14:32:42','14:33:52','14:35:02','14:35:50','14:36:10','14:36:45','14:38:30','14:40:04',
    '14:40:52','14:42:05','14:43:25','14:43:50','14:45:40','14:46:02','14:46:57','14:48:02','14:50:06','14:51:03',
    '14:52:10','14:53:12','14:53:30','14:54:44','14:56:10','14:57:23','14:58:42','14:59:36','15:00:19','15:00:52',
    '15:01:33','15:02:57','15:04:10','15:05:02','15:06:04','15:10:36','15:11:26','15:12:52','15:14:00','15:15:24',
    '15:16:01','15:17:04','15:19:36','15:22:07','15:22:54','15:24:04','15:24:54','15:25:39','15:26:02','15:27:35',
    '15:28:02','15:28:33','15:30:01','15:30:20','15:31:12','15:32:31','15:34:12','15:36:05','15:43:02','15:44:14',
    '15:45:55','15:46:39','15:47:55','15:48:54','15:49:58','15:51:18','15:51:48','15:54:00','15:54:46','15:56:07',
    '15:57:25','15:58:23','15:59:15','16:00:58'
]


# 10월 8일 (수) 2번 계산대

counter2_1008_service_interarrival_time = [
    '14:29:35','14:30:41','14:31:37','14:32:24','14:34:11','14:35:23','14:34:36','14:36:57','14:38:33','14:38:50',
    '14:40:04','14:40:19','14:43:36','14:43:57','14:45:29','14:45:52','14:47:19','14:48:50','14:49:40','14:50:22',
    '14:51:51','14:51:52','14:52:27','14:53:12','14:55:56','14:56:14','14:56:29','14:56:43','14:58:26','14:58:37',
    '14:59:40','15:02:09','15:03:02','15:03:34','15:04:29','15:09:54','15:11:10','15:12:20','15:15:16','15:17:19',
    '15:18:12','15:18:39','15:20:55','15:21:21','15:23:29','15:25:04','15:26:43','15:27:40','15:29:21','15:29:38',
    '15:29:55','15:31:44','15:32:37','15:33:01','15:34:03','15:35:17','15:36:16','15:37:24','15:38:20','15:39:06',
    '15:39:44','15:40:21','15:40:38','15:42:52','15:43:22','15:43:29','15:47:45','15:49:12','15:49:41','15:51:26',
    '15:52:57','15:54:57','15:57:06','15:57:53','15:59:16'
]



counter2_1008_wait_times = [
    '00:00:00','00:00:28','00:00:09','00:00:23','00:00:00','00:00:00','00:02:07','00:00:34','00:00:08','00:00:49',
    '00:00:43','00:01:16','00:00:00','00:00:51','00:00:14','00:00:26','00:00:00','00:00:00','00:00:15','00:00:33',
    '00:00:18','00:01:11','00:01:35','00:01:37','00:00:00','00:00:57','00:01:42','00:02:55','00:02:08','00:03:02',
    '00:02:37','00:01:02','00:00:50','00:01:03','00:01:04','00:00:00','00:00:09','00:00:11','00:00:00','00:00:00',
    '00:00:19','00:00:40','00:00:00','00:00:30','00:00:00','00:00:00','00:00:00','00:00:19','00:00:00','00:01:19',
    '00:02:08','00:01:04','00:01:14','00:01:14','00:02:05','00:01:58','00:02:23','00:02:21','00:02:11','00:02:19',
    '00:02:49','00:03:07','00:03:31','00:03:43','00:02:24','00:02:36','00:02:44','00:00:00','00:00:00','00:00:24',
    '00:00:19','00:00:11','00:00:00','00:00:00','00:00:19','00:00:38'
]

counter2_1008_service_times = [
    '00:01:34','00:00:37','00:01:01','00:01:17','00:00:42','00:01:20','00:00:48','00:01:10','00:00:58','00:01:08',
    '00:00:48','00:01:34','00:01:12','00:00:55','00:00:35','00:00:51','00:01:25','00:01:05','00:01:00','00:01:14',
    '00:00:54','00:00:59','00:00:47','00:00:32','00:01:15','00:01:00','00:01:27','00:00:56','00:01:05','00:00:38',
    '00:00:54','00:00:41','00:00:45','00:00:56','00:01:38','00:01:25','00:01:12','00:02:24','00:00:57','00:01:12',
    '00:00:48','00:01:08','00:00:56','00:01:30','00:01:28','00:01:18','00:01:16','00:00:58','00:01:36','00:01:06',
    '00:00:45','00:01:03','00:01:15','00:00:55','00:01:39','00:00:57','00:00:58','00:01:04','00:01:16','00:00:56',
    '00:01:01','00:00:29','00:00:55','00:00:42','00:00:15','00:01:09','00:01:06','00:01:06','00:00:53','00:01:40',
    '00:01:23','00:00:48','00:01:30','00:01:06','00:01:42','00:00:57'
]

counter2_1008_service_end_times = [
    '14:31:09','14:31:46','14:32:47','14:34:04','14:34:53','14:36:43','14:37:31','14:38:41','14:39:39','14:40:47',
    '14:41:35','14:43:09','14:44:48','14:45:43','14:46:18','14:47:09','14:48:44','14:49:55','14:50:55','14:52:09',
    '14:53:03','14:54:02','14:54:49','14:55:21','14:57:11','14:58:11','14:59:38','15:00:34','15:01:39','15:02:17',
    '15:03:11','15:03:52','15:04:37','15:05:33','15:07:11','15:11:19','15:12:31','15:14:55','15:16:13','15:18:31',
    '15:19:19','15:20:27','15:21:51','15:23:21','15:24:57','15:26:22','15:27:59','15:28:57','15:30:57','15:32:03',
    '15:32:48','15:33:51','15:35:06','15:36:01','15:37:40','15:38:37','15:39:35','15:40:39','15:41:55','15:42:51',
    '15:43:52','15:44:21','15:45:16','15:45:58','15:46:13','15:47:22','15:48:51','15:50:05','15:51:45','15:53:08',
    '15:53:56','15:56:27','15:58:12','15:59:54','16:00:51'
]

# 10월 15일(수) 1번 계산대(85개)


counter1_1015_service_interarrival_time = [
    '15:00:08','15:01:36','15:01:38','15:02:51','15:04:43','15:05:52','15:06:13','15:06:14','15:07:08','15:09:19',
    '15:10:36','15:11:20','15:11:50','15:12:11','15:13:11','15:15:08','15:15:20','15:16:02','15:16:50','15:17:59',
    '15:20:13','15:20:20','15:20:55','15:21:03','15:22:45','15:23:55','15:26:11','15:26:27','15:26:43','15:27:04',
    '15:27:43','15:28:15','15:29:57','15:31:31','15:31:32','15:31:35','15:32:02','15:33:05','15:34:09','15:34:14',
    '15:34:46','15:35:24','15:35:37','15:37:09','15:38:28','15:38:31','15:39:05','15:42:20','15:42:42','15:43:51',
    '15:44:32','15:45:09','15:46:00','15:46:02','15:46:35','15:48:43','15:50:02','15:52:34','15:52:41','15:53:02',
    '15:53:47','15:54:13','15:57:40','15:58:30','16:00:20','16:03:40','16:05:21','16:05:57','16:07:10','16:07:47',
    '16:11:37','16:14:24','16:15:13','16:15:58','16:16:30','16:17:28','16:18:43','16:20:28','16:20:56','16:22:03',
    '16:22:14','16:22:27','16:25:20','16:28:32','16:28:58'
]


counter1_1015_wait_times = [
    '00:00:00','00:00:00','00:01:04','00:02:11','00:00:46','00:01:48','00:01:44','00:02:18','00:02:30','00:00:40',
    '00:00:00','00:00:17','00:00:30','00:00:35','00:01:28','00:00:29','00:01:33','00:01:45','00:02:28','00:01:55',
    '00:00:33','00:01:53','00:02:29','00:03:04','00:02:10','00:01:29','00:00:00','00:00:57','00:01:21','00:01:59',
    '00:01:41','00:01:27','00:00:42','00:00:33','00:02:07','00:03:22','00:03:51','00:03:33','00:03:21','00:03:52',
    '00:04:02','00:04:51','00:06:03','00:04:56','00:04:45','00:05:11','00:05:11','00:02:52','00:03:21','00:03:03',
    '00:03:30','00:03:47','00:03:43','00:04:34','00:04:59','00:04:51','00:05:26','00:05:00','00:05:57','00:06:25',
    '00:06:55','00:07:01','00:04:10','00:04:49','00:05:03','00:02:14','00:01:37','00:01:56','00:01:44','00:03:34',
    '00:00:12','00:00:00','00:00:00','00:01:11','00:01:41','00:02:08','00:01:44','00:00:58','00:01:03','00:00:47',
    '00:01:00','00:01:24','00:00:00','00:00:00','00:00:27'
]

counter1_1015_service_times = [
    '00:00:18','00:01:06','00:02:20','00:00:27','00:02:11','00:00:17','00:00:35','00:01:06','00:00:21','00:00:31',
    '00:01:01','00:00:43','00:00:26','00:01:53','00:00:58','00:01:16','00:00:54','00:01:31','00:00:36','00:00:52',
    '00:01:27','00:01:11','00:00:43','00:00:48','00:00:29','00:00:23','00:01:13','00:00:40','00:00:59','00:00:21',
    '00:00:18','00:00:57','00:01:25','00:01:35','00:01:18','00:00:56','00:00:45','00:00:52','00:00:36','00:00:42',
    '00:01:27','00:01:25','00:00:26','00:01:08','00:00:29','00:00:34','00:00:56','00:00:51','00:00:51','00:01:08',
    '00:00:54','00:00:47','00:00:53','00:00:58','00:02:00','00:01:54','00:02:06','00:01:04','00:00:49','00:01:15',
    '00:00:32','00:00:36','00:01:29','00:02:04','00:00:31','00:01:04','00:00:55','00:01:01','00:02:27','00:00:28',
    '00:01:35','00:00:47','00:01:56','00:01:02','00:01:25','00:00:51','00:00:59','00:00:23','00:00:51','00:00:24',
    '00:00:37','00:00:24','00:01:08','00:00:53','00:00:51'
]

counter1_1015_service_end_times = [
    '15:00:26','15:02:42','15:05:02','15:05:29','15:07:40','15:07:57','15:08:32','15:09:38','15:09:59','15:10:30',
    '15:11:37','15:12:20','15:12:46','15:14:39','15:15:37','15:16:53','15:17:47','15:19:18','15:19:54','15:20:46',
    '15:22:13','15:23:24','15:24:07','15:24:55','15:25:24','15:25:47','15:27:24','15:28:04','15:29:03','15:29:24',
    '15:29:42','15:30:39','15:32:04','15:33:39','15:34:57','15:35:53','15:36:38','15:37:30','15:38:06','15:38:48',
    '15:40:15','15:41:40','15:42:06','15:43:13','15:43:42','15:44:16','15:45:12','15:46:03','15:46:54','15:48:02',
    '15:48:56','15:49:43','15:50:36','15:51:34','15:53:34','15:55:28','15:57:34','15:58:38','15:59:27','16:00:42',
    '16:01:14','16:01:50','16:03:19','16:05:23','16:05:54','16:06:58','16:07:53','16:08:54','16:11:21','16:11:49',
    '16:13:24','16:15:11','16:17:09','16:18:11','16:19:36','16:20:27','16:21:26','16:21:49','16:22:50','16:23:14',
    '16:23:51','16:24:15','16:26:28','16:29:25','16:30:16'
]

# 10월 15일 (수) 2번 계산대 (76개)

counter2_1015_service_interarrival_time= [
    '15:00:55','15:02:30','15:03:36','15:04:07','15:04:40','15:05:16','15:11:33','15:11:49','15:13:49','15:15:18',
    '15:16:08','15:17:28','15:18:11','15:18:58','15:19:17','15:20:33','15:20:58','15:21:21','15:23:13','15:24:37',
    '15:25:10','15:27:10','15:27:42','15:28:36','15:30:07','15:31:04','15:31:15','15:31:35','15:32:26','15:33:33',
    '15:33:58','15:34:07','15:34:15','15:35:02','15:36:16','15:37:35','15:38:31','15:39:04','15:39:38','15:40:00',
    '15:41:18','15:43:07','15:44:11','15:45:19','15:45:44','15:46:02','15:46:19','15:48:02','15:49:37','15:50:40',
    '15:52:36','15:53:00','15:55:30','15:57:09','15:58:46','16:00:01','16:02:30','16:02:40','16:05:37','16:07:30',
    '16:07:41','16:10:36','16:14:44','16:14:56','16:15:40','16:16:25','16:17:01','16:18:33','16:19:41','16:21:06',
    '16:21:46','16:23:01','16:24:54','16:25:07','16:28:16','16:28:41'
]

counter2_1015_wait_times = [
    '00:01:47','00:00:45','00:00:11','00:00:26','00:01:10','00:02:26','00:02:15','00:02:35','00:01:22','00:01:04',
    '00:01:00','00:00:31','00:00:38','00:00:56','00:03:06','00:02:57','00:03:09','00:03:05','00:01:30','00:01:54',
    '00:02:40','00:01:47','00:01:44','00:01:24','00:01:19','00:01:01','00:01:49','00:02:12','00:02:22','00:02:21',
    '00:03:07','00:03:48','00:03:59','00:04:21','00:04:10','00:03:22','00:03:02','00:03:33','00:04:04','00:04:33',
    '00:03:36','00:02:13','00:04:06','00:03:20','00:03:41','00:04:15','00:07:33','00:06:50','00:05:47','00:06:10',
    '00:05:01','00:05:10','00:03:08','00:02:30','00:01:57','00:01:48','00:00:42','00:01:15','00:01:45','00:01:11',
    '00:02:37','00:00:30','00:00:34','00:00:37','00:01:09','00:01:15','00:00:51','00:01:02','00:01:03','00:01:15',
    '00:01:12','00:01:25','00:01:07','00:01:36','00:00:00','00:00:18'
]

counter2_1015_service_times = [
    '00:00:18','00:00:30','00:00:28','00:01:07','00:01:07','00:06:05','00:00:34','00:00:38','00:01:09','00:00:39',
    '00:00:49','00:00:36','00:01:02','00:02:24','00:00:57','00:00:30','00:00:12','00:00:14','00:01:44','00:01:02',
    '00:00:53','00:00:19','00:00:29','00:00:57','00:00:33','00:00:45','00:00:16','00:00:58','00:00:59','00:01:07',
    '00:00:41','00:00:16','00:01:01','00:00:48','00:00:24','00:00:34','00:00:56','00:00:58','00:00:43','00:00:19',
    '00:00:14','00:02:30','00:00:14','00:00:34','00:00:50','00:03:32','00:00:57','00:00:27','00:00:06','00:00:43',
    '00:00:21','00:00:20','00:00:49','00:00:57','00:01:04','00:01:01','00:00:32','00:00:53','00:01:17','00:01:26',
    '00:00:41','00:00:41','00:00:13','00:01:07','00:00:49','00:00:04','00:01:33','00:01:05','00:01:32','00:00:19',
    '00:01:14','00:01:10','00:00:38','00:00:41','00:00:39','00:01:20'
]

counter2_1015_service_end_times = [
    '15:03:00','15:03:45','15:04:15','15:05:40','15:06:57','15:13:47','15:14:22','15:15:02','15:16:20','15:17:01',
    '15:17:57','15:18:35','15:19:51','15:22:18','15:23:20','15:24:00','15:24:19','15:24:40','15:26:27','15:27:33',
    '15:28:43','15:29:16','15:29:55','15:30:57','15:31:59','15:32:50','15:33:20','15:34:45','15:35:47','15:37:01',
    '15:37:46','15:38:11','15:39:15','15:40:11','15:40:50','15:41:31','15:42:29','15:43:35','15:44:25','15:44:52',
    '15:45:08','15:47:50','15:48:31','15:49:13','15:50:15','15:53:49','15:54:49','15:55:19','15:55:30','15:57:33',
    '15:57:58','15:58:30','15:59:27','16:00:36','16:01:47','16:02:50','16:03:44','16:04:48','16:08:39','16:10:07',
    '16:10:59','16:11:47','16:15:31','16:16:40','16:17:38','16:17:44','16:19:25','16:20:40','16:22:16','16:22:40',
    '16:24:12','16:25:36','16:26:39','16:27:24','16:28:55','16:30:19'
]

